<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MC_Tiempo_AyC_vs_CRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import timeit

In [9]:
def g(x):
  return np.sqrt(np.arctan(x))

Monte Carlo: Acierto y Error

In [12]:
def estimacion_1(n):
  aciertos=0
  for i in range (n):
    x = rd.random()
    y = rd.random()

    if y <= g(x):
      aciertos+=1
  I=aciertos/n
  return I

In [13]:
estimacion_1(1000)

0.633

Hacer una función que la llame 50 veces, de ahí calcular la varianza y de ahí medir el tiempo.

In [5]:
%%time
aciertos=0
N=10000

for i in range (N):
  x = rd.random()
  y = rd.random()

  if y <= g(x):
    aciertos+=1
  else:
    aciertos+=0

I=aciertos/N
var=(aciertos/N)-(I**2)
print("El valor estimado es: ",I, "\nVarianza: ",var)

El valor estimado es:  0.6309 
Varianza:  0.23286519
CPU times: user 19.1 ms, sys: 0 ns, total: 19.1 ms
Wall time: 19.1 ms


Monte Carlo: Crudo

In [10]:
%%time

def crudo(N):
  G=[]
  for i in range (N):
    u=rd.random()
    G.append(g(u))
  return np.mean(G), np.var(G), np.std(G), G

e=crudo(10000)

print("El valor estimado es: ", e[0], "\nVarianza: ", e[1])

El valor estimado es:  0.6323026149264287 
Varianza:  0.04227909022855156
CPU times: user 18.5 ms, sys: 987 µs, total: 19.5 ms
Wall time: 19.3 ms


Tiempos de ejecución:

In [4]:
# Obtener los tiempos de ejecución y varianzas de las celdas anteriores
# Para el método de Acierto y Error (celda 87Tm8hdlQT6r)
t1 = 28.6  # Tiempo en ms
var1 = var

# Para el método Crudo (celda WdPbAqZ5SQ5y)
t2 = 24.3  # Tiempo en ms
var2 = e

# Calcular la comparación usando la fórmula E = ((t_1 * var_1) / (t_2 * var_2))
E = (t1 * var1) / (t2 * var2)

print(f"Tiempo de ejecución del método Acierto y Error (t1): {t1} ms")
print(f"Varianza del método Acierto y Error (var1): {var1}")
print(f"Tiempo de ejecución del método Crudo (t2): {t2} ms")
print(f"Varianza del método Crudo (var2): {var2}")
print(f"\nValor de E = ((t1 * var1) / (t2 * var2)): {E}")

if E < 1:
  print("Según la fórmula, el método Crudo es más eficiente.")
else:
  print("Según la fórmula, el método Acierto y Error es más eficiente o similar.")

Tiempo de ejecución del método Acierto y Error (t1): 28.6 ms
Varianza del método Acierto y Error (var1): 0.23294364
Tiempo de ejecución del método Crudo (t2): 24.3 ms
Varianza del método Crudo (var2): 0.042286407327671666

Valor de E = ((t1 * var1) / (t2 * var2)): 6.483504672830493
Según la fórmula, el método Acierto y Error es más eficiente o similar.
